<img src="static/pybofractal.png" alt="Pybonacci" style="width: 200px;"/>
<img src="static/cacheme_logo.png" alt="CAChemE" style="width: 300px;"/>

# The Transport Problem

> **Note:** Adapted from https://github.com/Pyomo/PyomoGallery, see LICENSE.BSD

## Summary

The goal of the Transport Problem is to select the quantities of an homogeneous good that has several production plants and several punctiform markets as to minimise the transportation costs.

It is the default tutorial for the GAMS language, and GAMS equivalent code is inserted as single-dash comments. The original GAMS code needs slighly different ordering of the commands and it's available at http://www.gams.com/mccarl/trnsport.gms.

## Problem Statement

The Transport Problem can be formulated mathematically as a linear programming problem using the following model.  

### Sets

 $I$ = set of canning plants  
 $J$ = set of markets

### Parameters

 $a_i$     = capacity of plant $i$ in cases, $\forall i \in I$ <br />
 $b_j$     = demand at market $j$ in cases, $\forall j \in J$ <br />
 $d_{i,j}$ = distance in thousands of miles, $\forall i \in I, \forall j \in J$ <br />
 $f$       = freight in dollars per case per thousand miles <br />
 $c_{i,j}$ = transport cost in thousands of dollars per case
 
 $c_{i,j}$ is obtained exougenously to the optimisation problem as $c_{i,j} = f \cdot d_{i,j}$, $\forall i \in I, \forall j \in J$
 
### Variables
 $x_{i,j}$ = shipment quantities in cases <br />
 z         = total transportation costs in thousands of dollars

### Objective

Minimize the total cost of the shipments: <br />
$\min_{x} z = \sum_{i \in I} \sum_{j \in J} c_{i,j} x_{i,j}$

### Constraints


Observe supply limit at plant i: <br />
 $\sum_{i \in I} x_{i,j} \leq a_{i}$, $\forall i \in I$
 
Satisfy demand at market j: <br />
 $\sum_{j \in J} x_{i,j} \geq b_{j}$, $\forall j \in J$

Non-negative transportation quantities <br />
 $x_{i,j} \geq 0$, $\forall i \in I, \forall j \in J$

## Pyomo Formulation

### Creation of the Model

In pyomo everything is an object. The various components of the model (sets, parameters, variables, constraints, objective..) are all attributes of the main model object while being objects themselves.

There are two type of models in pyomo: A `ConcreteModel` is one where all the data is defined at the model creation. We are going to use this type of model in this tutorial. Pyomo however supports also an `AbstractModel`, where the model structure is firstly generated and then particular instances of the model are generated with a particular set of data.

The first thing to do in the script is to load the pyomo library and create a new `ConcreteModel` object. We have little imagination here, and we call our model "model". You can give it whatever name you want. However, if you give your model an other name, you also need to create a `model` object at the end of your script:

### Set Definitions

Sets are created as attributes object of the main model objects and all the information is given as parameter in the constructor function. Specifically, we are passing to the constructor the initial elements of the set and a documentation string to keep track on what our set represents: 

In [1]:
## Define sets ##
#  Sets
#       i   canning plants   / seattle, san-diego /
#       j   markets          / new-york, chicago, topeka / ;

### Parameters

Parameter objects are created specifying the sets over which they are defined and are initialised with either a python dictionary or a scalar: 

In [2]:
## Define parameters ##
#   Parameters
#       a(i)  capacity of plant i in cases
#         /    seattle     350
#              san-diego   600  /
#       b(j)  demand at market j in cases
#         /    new-york    325
#              chicago     300
#              topeka      275  / ;


#  Table d(i,j)  distance in thousands of miles
#                    new-york       chicago      topeka
#      seattle          2.5           1.7          1.8
#      san-diego        2.5           1.8          1.4  ;


#  Scalar f  freight in dollars per case per thousand miles  /90/ ;



A third, powerful way to initialize a parameter is using a user-defined function.

This function will be automatically called by pyomo with any possible (i,j) set. In this case pyomo will actually call `c_init()` six times in order to initialize the `model.c` parameter. 

In [4]:
#  Parameter c(i,j)  transport cost in thousands of dollars per case ;
#            c(i,j) = f * d(i,j) / 1000 ;



### Variables

Similar to parameters, variables are created specifying their domain(s). For variables we can also specify the upper/lower bounds in the constructor.

Differently from GAMS, we don't need to define the variable that is on the left hand side of the objective function. 

In [5]:
## Define variables ##
#  Variables
#       x(i,j)  shipment quantities in cases
#       z       total transportation costs in thousands of dollars ;
#  Positive Variable x ;



### Constrains

At this point, it should not be a surprise that constrains are again defined as model objects with the required information passed as parameter in the constructor function. 

In [7]:
## Define contrains ##
# supply(i)   observe supply limit at plant i
# supply(i) .. sum (j, x(i,j)) =l= a(i)

# demand(j)   satisfy demand at market j ;  
# demand(j) .. sum(i, x(i,j)) =g= b(j);


The above code take advantage of [list comprehensions](https://docs.python.org/2/tutorial/datastructures.html#list-comprehensions), a powerful feature of the python language that provides a concise way to loop over a list. If we take the supply_rule as example, this is actually called two times by pyomo (once for each of the elements of i). Without list comprehensions we would have had to write our function using a for loop, like: 

Using list comprehension is however quicker to code and more readable. 

### Objective and Solving

The definition of the objective is similar to those of the constrains, except that most solvers require a scalar objective function, hence a unique function, and we can specify the sense (direction) of the optimisation. 

In [8]:
## Define Objective and solve ##
#  cost        define objective function
#  cost ..        z  =e=  sum((i,j), c(i,j)*x(i,j)) ;
#  Model transport /all/ ;
#  Solve transport using lp minimizing z ;



As we are here looping over two distinct sets, we can see how list comprehension really simplifies the code. The objective function could have being written without list comprehension as: 

### Retrieving the Output

We use the `pyomo_postprocess()` function to retrieve the output and do something with it.  For example, we could display solution values (see below), plot a graph with [matplotlib](http://matplotlib.org/) or save it in a csv file.

This function is called by pyomo after the solver has finished. 

In [9]:
## Display of the output ##
# Display x.l, x.m ;



We can print model structure information with `model.pprint()` (“pprint” stand for “pretty print”).
Results are also by default saved in a `results.json` file or, if PyYAML is installed in the system, in `results.yml`.

### Editing and Running the Script

Differently from GAMS, you can use whatever editor environment you wish to code a pyomo script. If you don't need debugging features, a simple text editor like Notepad++ (in windows), gedit or kate (in Linux) will suffice. They already have syntax highlight for python.

If you want advanced features and debugging capabilities you can use a dedicated Python IDE, like e.g. Spyder.

You will normally run the script as `pyomo solve –solver=glpk transport.py`. You can output solver specific output adding the option `–stream-output`. If you want to run the script as `python transport.py` add the following lines at the end:

## Complete script

Here is the complete script: 

## Solutions
Running the model lead to the following output:

In [11]:
!pyomo solve --solver=glpk transport.py

By default, the optimization results are stored in the file `results.json`:

This solution shows that the minimum transport costs is attained when 300 cases are sent from the Seattle plant to the Chicago market, 50 cases from Seattle to New-York and 275 cases each are sent from San-Diego plant to New-York and Topeka markets.

The total transport costs will be $153,675.

## References

* Original problem formulation:
  - Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions. Princeton University Press, Princeton, New Jersey, 1963.
* GAMS implementation:
  - Rosenthal, R E, Chapter 2: A GAMS Tutorial. In GAMS: A User's Guide. The Scientific Press, Redwood City, California, 1988.
* Pyomo translation: Antonello Lobianco